In [2]:
## This contains a simple IK problem for solo 12
## Author : Avadesh Meduri
## Date : 25/02/2021

import time
import crocoddyl
import numpy as np
import pinocchio as pin

from robot_properties_solo.config import Solo12Config
from py_biconvex_mpc.ik.inverse_kinematics import InverseKinematics

solo import 


In [3]:
robot = Solo12Config.buildRobotWrapper()
robot_model = robot.model
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=True)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [4]:
# Running and terminal action models
dt = 1e-3
T = 0.2
N = int(T/dt)

In [5]:
traj = []
for t in range(N):
    omega = np.pi/N
    x = 0.15*np.sin(2*omega*t)
    traj.append(np.array([0.4 + x , -0.16 ,0.1*np.sin(omega*t)]))
    
traj_hl = traj + np.array([-.4, 0.32, 0])

In [6]:
ori_traj = []
com_traj = np.zeros((N,3)) # center of mass traj
com_traj[:,2] = 0.25
com_traj[:,1] = np.linspace(0,1.0, N)
# print(com_traj[:,0])
# for t in range(N):
#     omega = np.pi/N
#     rz = (np.pi/5.0)*np.sin(2*omega*t)
#     ori_traj.append(pin.utils.rpyToMatrix(rz,0.0,0.0))

In [7]:
cmom_traj = np.zeros((N,6)) #centroidal momentum traj
cmom_traj[:,2] = -2.0
vel_traj = np.zeros((N, 6))

In [8]:
# setting up running cost model for swing foot task
ik = InverseKinematics(robot_model, dt, T)

# ik.add_com_position_tracking_task(0, T, com_traj, 1e5, "com_task")
# ik.add_orientation_tracking_task(robot_model.getFrameId("base_link"), 0, T, ori_traj, 1e4, "base_otc")
# ik.add_centroidal_momentum_tracking_task(0, T, cmom_traj, 1e5, "cent_tc")

ik.add_position_tracking_task(robot_model.getFrameId("FR_FOOT"), 0, T, traj, 1e3, "FR_ftc")
# ik.add_position_tracking_task(robot_model.getFrameId("HL_FOOT"), 0, T, traj_hl, 1e3, "HL_ftc")

# ik.add_velocity_tracking_task(robot_model.getFrameId("FL_FOOT"), 0, T, vel_traj, 1e3, "FL_vtc")
# ik.add_velocity_tracking_task(robot_model.getFrameId("FR_FOOT"), 0, T, vel_traj, 1e3, "FR_vtc")
ik.add_velocity_tracking_task(robot_model.getFrameId("HL_FOOT"), 0, T, vel_traj, 1e3, "HL_vtc")
# ik.add_velocity_tracking_task(robot_model.getFrameId("HR_FOOT"), 0, T, vel_traj, 1e3, "HR_vtc")

ik.add_state_regularization_cost(0, T, 1e-4, "xReg")
ik.add_ctrl_regularization_cost(0, T, 1e-7, "uReg")

# setting up terminal cost model
xRegCost = crocoddyl.CostModelState(ik.state)
uRegCost = crocoddyl.CostModelControl(ik.state)
comTrack = crocoddyl.CostModelCoMPosition(ik.state, com_traj[-1], ik.state.nv)

ik.terminalCostModel.addCost("stateReg", xRegCost, 1e-4)
ik.terminalCostModel.addCost("ctrlReg", uRegCost, 1e-7) 
# ik.terminalCostModel.addCost("comtask", comTrack, 1e+5)

In [9]:
ik.setup_costs()

q0 = np.array(Solo12Config.initial_configuration)
x0 = np.concatenate([q0, pin.utils.zero(robot_model.nv)])
xs = ik.optimize(x0) 


In [10]:
for i in range(len(xs)):
    time.sleep(0.01)
    viz.display(xs[i][:robot_model.nq])

In [ ]:
for i in range(len(xs)):
    q = xs[i][:robot_model.nq]
    v = xs[i][robot_model.nq:]
    pin.forwardKinematics(robot.model, robot.data, q, v)
#     pin.updateFramePlacements(robot.model, robot.data)
    print(pin.centerOfMass(robot.model, robot.data, q, v))
    pin.computeCentroidalMomentum(robot.model, robot.data)
#     print(np.round(robot.data.hg, 2))